In [1]:
from pyspark.sql.window import Window
from pyspark.sql import functions as F

In [2]:
list=[[3,'B'],
     [1,'A'],
     [2,'A'],
     [5,'C'],
     [4,'A']]


Main=spark.createDataFrame(list,['H1','H2'])
Main.show()

list1=[[1,'A',0.5],
      [4,'A',0.3],
      [2,'D',0.4],
      [3,'A',0.6]]

Table2= spark.createDataFrame(list1,['H1','H2','R'])
Table2.show()


list2=[['A',0.4],
      ['B',0.6],
      ['C',0.2],
      ['D',0.4],
      ['E',0.9],
      ['F',0.1],
      ['G',0.3]]

Table3= spark.createDataFrame(list2,['H2','R'])
Table3.show()


+---+---+
 H1| H2|
+---+---+
 3| B|
 1| A|
 2| A|
 5| C|
 4| A|
+---+---+

+---+---+---+
 H1| H2| R|
+---+---+---+
 1| A|0.5|
 4| A|0.3|
 2| D|0.4|
 3| A|0.6|
+---+---+---+

+---+---+
 H2| R|
+---+---+
 A|0.4|
 B|0.6|
 C|0.2|
 D|0.4|
 E|0.9|
 F|0.1|
 G|0.3|
+---+---+

In [3]:
Main.join(Table2.withColumnRenamed("H2","H22"), ['H1'],'left').drop("H22")\
    .join(Table3.withColumnRenamed("R","R2"),['H2'])\
    .withColumn("R", F.when(F.col("R").isNull(), F.col("R2")).otherwise(F.col("R"))).drop("R2").show()

+---+---+---+
 H2| H1| R|
+---+---+---+
 B| 3|0.6|
 C| 5|0.2|
 A| 1|0.5|
 A| 2|0.4|
 A| 4|0.3|
+---+---+---+

In [4]:
list=[['grass','poison'],
      ['grass','poison'],
      ['grass','poison'],
      ['fire',None],
      ['fire',None],
      ['fire','flying'],
      ['water',None],
      ['water',None],
      ['water',None],
      ['bug',None],
      ['bug',None],
      ['bug',None],
      ['flying','fire']]

df=spark.createDataFrame(list,['type1','type2'])

df.show()

+------+------+
 type1| type2|
+------+------+
 grass|poison|
 grass|poison|
 grass|poison|
 fire| null|
 fire| null|
 fire|flying|
 water| null|
 water| null|
 water| null|
 bug| null|
 bug| null|
 bug| null|
flying| fire|
+------+------+

In [5]:
df.dropna().distinct().show()

+------+------+
 type1| type2|
+------+------+
flying| fire|
 fire|flying|
 grass|poison|
+------+------+

In [6]:
list=[['PT381201021','2019-08-22','Albumin','2019-08-14','4.3',8],
      ['PT381201021','2019-05-17','Albumin','NA','NA',0],
      ['PT381201021','2019-05-18','Albumin','NA','NA',0],
      ['PT381201021','2019-05-21','Albumin','NA','NA',0],
      ['PT381201021','2019-05-23','Albumin','NA','NA',0],
      ['PT381201021','2019-05-16','Albumin','NA','NA',0],
      ['PT381201021','2019-05-19','Albumin','NA','NA',0],
      ['PT381201021','2019-05-22','Albumin','NA','NA',0],
      ['PT381201021','2019-05-20','Albumin','NA','NA',0]]


df=spark.createDataFrame(list,['ptid','blast_date','test_name','result_date','test_result','date_diff'])
df.show()

+-----------+----------+---------+-----------+-----------+---------+
 ptid|blast_date|test_name|result_date|test_result|date_diff|
+-----------+----------+---------+-----------+-----------+---------+
PT381201021|2019-08-22| Albumin| 2019-08-14| 4.3| 8|
PT381201021|2019-05-17| Albumin| NA| NA| 0|
PT381201021|2019-05-18| Albumin| NA| NA| 0|
PT381201021|2019-05-21| Albumin| NA| NA| 0|
PT381201021|2019-05-23| Albumin| NA| NA| 0|
PT381201021|2019-05-16| Albumin| NA| NA| 0|
PT381201021|2019-05-19| Albumin| NA| NA| 0|
PT381201021|2019-05-22| Albumin| NA| NA| 0|
PT381201021|2019-05-20| Albumin| NA| NA| 0|
+-----------+----------+---------+-----------+-----------+---------+

In [7]:
w=Window().partitionBy("ptid","test_name").orderBy(F.to_timestamp("blast_date","yyyy-MM-dd").cast("long")).rangeBetween(Window.currentRow,86400*91)

df.withColumn("collect", F.collect_list(F.array("result_date","test_result")).over(w))\
  .withColumn("collect", F.expr("""filter(collect,x-> array_contains(x,'NA')!=True)""")[0])\
  .withColumn("result_date", F.when((F.col("result_date")=='NA')&(F.col("collect").isNotNull()),F.col("collect")[0]).otherwise(F.col("result_date")))\
  .withColumn("test_result", F.when((F.col("test_result")=='NA')&(F.col("collect").isNotNull()),F.col("collect")[1]).otherwise(F.col("test_result"))).drop("timestamp","collect").show(truncate=False)

+-----------+----------+---------+-----------+-----------+---------+
ptid |blast_date|test_name|result_date|test_result|date_diff|
+-----------+----------+---------+-----------+-----------+---------+
PT381201021|2019-05-16|Albumin |NA |NA |0 |
PT381201021|2019-05-17|Albumin |NA |NA |0 |
PT381201021|2019-05-18|Albumin |NA |NA |0 |
PT381201021|2019-05-19|Albumin |NA |NA |0 |
PT381201021|2019-05-20|Albumin |NA |NA |0 |
PT381201021|2019-05-21|Albumin |NA |NA |0 |
PT381201021|2019-05-22|Albumin |NA |NA |0 |
PT381201021|2019-05-23|Albumin |2019-08-14 |4.3 |0 |
PT381201021|2019-08-22|Albumin |2019-08-14 |4.3 |8 |
+-----------+----------+---------+-----------+-----------+---------+

In [8]:
TRAIT_0 = 0
TRAIT_1 = 1
TRAIT_2 = 2
def flag_to_list(flag):
    trait_list = []
    if flag & (1 << TRAIT_0 ):
        trait_list.append("TRAIT_0")
    elif flag & (1 << TRAIT_1):
        trait_list.append("TRAIT_1")
    elif flag & (1 << TRAIT_2):
        trait_list.append("TRAIT_2")

    return trait_list
  
flag_to_list(1)

Out[63]: ['TRAIT_0']

In [9]:
1<<TRAIT_1

Out[65]: 2

In [10]:
list=[['val1','val2','val3','val4','val5']]

df=spark.createDataFrame(list,['col1','col2','col3','col4','col5'])

df.show()

+----+----+----+----+----+
col1|col2|col3|col4|col5|
+----+----+----+----+----+
val1|val2|val3|val4|val5|
+----+----+----+----+----+

In [11]:
[(F.lit(x),F.col(x)) for x in df.columns]

Out[52]: [(Column<b'col1'>, Column<b'col1'>),
 (Column<b'col2'>, Column<b'col2'>),
 (Column<b'col3'>, Column<b'col3'>),
 (Column<b'col4'>, Column<b'col4'>),
 (Column<b'col5'>, Column<b'col5'>)]

In [12]:
df.withColumn("mapped", F.create_map(*[F.array(F.lit(x),F.col(x)) for x in df.columns]))\
  .select(F.explode("mapped")).show()

--------------------------------------------------------------------------- 
 Py4JJavaError Traceback (most recent call last)
 /databricks/spark/python/pyspark/sql/utils.py in deco (*a, **kw) 
 62 try : 
 ---> 63 return f ( * a , ** kw ) 
 64 except py4j . protocol . Py4JJavaError as e : 

 /databricks/spark/python/lib/py4j-0.10.7-src.zip/py4j/protocol.py in get_return_value (answer, gateway_client, target_id, name) 
 327 "An error occurred while calling {0}{1}{2}.\n" . 
 --> 328 format(target_id, ".", name), value)
 329 else : 

 Py4JJavaError : An error occurred while calling o2312.withColumn.
: org.apache.spark.sql.AnalysisException: cannot resolve 'map(array('col1', `col1`), array('col2', `col2`), array('col3', `col3`), array('col4', `col4`), array('col5', `col5`))' due to data type mismatch: map expects a positive even number of arguments.;;
'Project [col1#2964, col2#2965, col3#2966, col4#2967, col5#2968, map(array(col1, col1#2964), array(col2, col2#2965), array(col3, col3#2966), array(col4, col4#2967), array(col5, col5#2968)) AS mapped#3017]
+- LogicalRDD [col1#2964, col2#2965, col3#2966, col4#2967, col5#2968], false

	at org.apache.spark.sql.catalyst.analysis.package$AnalysisErrorAt.failAnalysis(package.scala:42)
	at org.apache.spark.sql.catalyst.analysis.CheckAnalysis$$anonfun$checkAnalysis$1$$anonfun$apply$3.applyOrElse(CheckAnalysis.scala:126)
	at org.apache.spark.sql.catalyst.analysis.CheckAnalysis$$anonfun$checkAnalysis$1$$anonfun$apply$3.applyOrElse(CheckAnalysis.scala:111)
	at org.apache.spark.sql.catalyst.trees.TreeNode$$anonfun$6.apply(TreeNode.scala:303)
	at org.apache.spark.sql.catalyst.trees.TreeNode$$anonfun$6.apply(TreeNode.scala:303)
	at org.apache.spark.sql.catalyst.trees.CurrentOrigin$.withOrigin(TreeNode.scala:76)
	at org.apache.spark.sql.catalyst.trees.TreeNode.transformUp(TreeNode.scala:302)
	at org.apache.spark.sql.catalyst.trees.TreeNode$$anonfun$5.apply(TreeNode.scala:300)
	at org.apache.spark.sql.catalyst.trees.TreeNode$$anonfun$5.apply(TreeNode.scala:300)
	at org.apache.spark.sql.catalyst.trees.TreeNode$$anonfun$8.apply(TreeNode.scala:353)
	at org.apache.spark.sql.catalyst.trees.TreeNode.mapProductIterator(TreeNode.scala:207)
	at org.apache.spark.sql.catalyst.trees.TreeNode.mapChildren(TreeNode.scala:351)
	at org.apache.spark.sql.catalyst.trees.TreeNode.transformUp(TreeNode.scala:300)
	at org.apache.spark.sql.catalyst.plans.QueryPlan$$anonfun$transformExpressionsUp$1.apply(QueryPlan.scala:94)
	at org.apache.spark.sql.catalyst.plans.QueryPlan$$anonfun$transformExpressionsUp$1.apply(QueryPlan.scala:94)
	at org.apache.spark.sql.catalyst.plans.QueryPlan$$anonfun$3.apply(QueryPlan.scala:106)
	at org.apache.spark.sql.catalyst.plans.QueryPlan$$anonfun$3.apply(QueryPlan.scala:106)
	at org.apache.spark.sql.catalyst.trees.CurrentOrigin$.withOrigin(TreeNode.scala:76)
	at org.apache.spark.sql.catalyst.plans.QueryPlan.transformExpression$1(QueryPlan.scala:105)
	at org.apache.spark.sql.catalyst.plans.QueryPlan.org$apache$spark$sql$catalyst$plans$QueryPlan$$recursiveTransform$1(QueryPlan.scala:117)
	at org.apache.spark.sql.catalyst.plans.QueryPlan$$anonfun$org$apache$spark$sql$catalyst$plans$QueryPlan$$recursiveTransform$1$2.apply(QueryPlan.scala:122)
	at scala.collection.TraversableLike$$anonfun$map$1.apply(TraversableLike.scala:234)
	at scala.collection.TraversableLike$$anonfun$map$1.apply(TraversableLike.scala:234)
	at scala.collection.immutable.List.foreach(List.scala:392)
	at scala.collection.TraversableLike$class.map(TraversableLike.scala:234)
	at scala.collection.immutable.List.map(List.scala:296)
	at org.apache.spark.sql.catalyst.plans.QueryPlan.org$apache$spark$sql$catalyst$plans$QueryPlan$$recursiveTransform$1(QueryPlan.scala:122)
	at org.apache.spark.sql.catalyst.plans.QueryPlan$$anonfun$4.apply(QueryPlan.scala:127)
	at org.apache.spark.sql.catalyst.trees.TreeNode.mapProductIterator(TreeNode.scala:207)
	at org.apache.spark.sql.catalyst.plans.QueryPlan.mapExpressions(QueryPlan.scala:127)
	at org.apac

In [13]:
list=[[1, 'A', '01/20/2020'],
       [1, 'B', '01/25/2020'],
       [1, 'C', '01/25/2020'],
       [1, 'A', '02/20/2020'],
       [1, 'B', '02/25/2020'],
       [1, 'C', 'NA']]

df=spark.createDataFrame(list,['id','test','date'])


df.show()

+---+----+----------+
 id|test| date|
+---+----+----------+
 1| A|01/20/2020|
 1| B|01/25/2020|
 1| C|01/25/2020|
 1| A|02/20/2020|
 1| B|02/25/2020|
 1| C| NA|
+---+----+----------+

In [14]:
w=Window().partitionBy("id","test").orderBy(F.to_date("date","MM/dd/yyyy").desc())

df.withColumn("lag", F.lag("date").over(w)).show()


+---+----+----------+----------+
 id|test| date| lag|
+---+----+----------+----------+
 1| C|01/25/2020| null|
 1| C| NA|01/25/2020|
 1| B|02/25/2020| null|
 1| B|01/25/2020|02/25/2020|
 1| A|02/20/2020| null|
 1| A|01/20/2020|02/20/2020|
+---+----+----------+----------+

In [15]:
list=[['Bulbasaur','grass',49],
      ['Ivysaur','grass',62],
      ['Venusaur','grass',100],
      ['Charmander','fire',52],
      ['Charmeleon','fire',64],
      ['Charizard','fire',104],
      ['Squirtle','water',48],
      ['Wartortle','water',63]]

df=spark.createDataFrame(list,['name','type1','attack'])

df.show()

+----------+-----+------+
 name|type1|attack|
+----------+-----+------+
 Bulbasaur|grass| 49|
 Ivysaur|grass| 62|
 Venusaur|grass| 100|
Charmander| fire| 52|
Charmeleon| fire| 64|
 Charizard| fire| 104|
 Squirtle|water| 48|
 Wartortle|water| 63|
+----------+-----+------+

In [16]:
from pyspark.sql import functions as F
from pyspark.sql.window import Window
w=Window().partitionBy("type1")
df.withColumn("max",F.max("attack").over(w)).filter('attack=max').select("name").show()

+---------+
 name|
+---------+
 Venusaur|
Wartortle|
Charizard|
+---------+

In [17]:
df.groupBy("type1").agg(F.max("attack").alias("attack")).show()

+-----+------+
type1|attack|
+-----+------+
grass| 100|
water| 63|
 fire| 104|
+-----+------+

In [18]:
import pandas as pd
import datetime

data = {'date': ['2014-01-01', '2014-01-02', '2014-01-03', '2014-01-04', '2014-01-05', '2014-01-06'],
     'customerid': [2, 2, 2, 3, 4, 3], 'names': ['Andrew', 'Pete', 'Sean', 'Steve', 'Ray', 'Stef'], 'PaymentType': ['OI', 'CC', 'CC', 'OI', 'OI', 'OI']}
data = pd.DataFrame(data)
data['date'] = pd.to_datetime(data['date'])
spark_data= spark.createDataFrame(data)

In [19]:
win = Window().partitionBy('customerid').orderBy((F.col('date')).cast("long")).rangeBetween(-(2*86400), Window.currentRow)

spark_data.withColumn("names_array",\
                      F.collect_list(F.when(F.col("PaymentType")=='OI',F.col("names"))\
                      .otherwise(F.lit(None))).over(win)).sort(F.col("date").asc()).show()

+-------------------+----------+------+-----------+-------------+
 date|customerid| names|PaymentType| names_array|
+-------------------+----------+------+-----------+-------------+
2014-01-01 00:00:00| 2|Andrew| OI| [Andrew]|
2014-01-02 00:00:00| 2| Pete| CC| [Andrew]|
2014-01-03 00:00:00| 2| Sean| CC| [Andrew]|
2014-01-04 00:00:00| 3| Steve| OI| [Steve]|
2014-01-05 00:00:00| 4| Ray| OI| [Ray]|
2014-01-06 00:00:00| 3| Stef| OI|[Steve, Stef]|
+-------------------+----------+------+-----------+-------------+

In [20]:
date      |customerid|PaymentType|names_array|
2014-01-01|2         |OI         |['Andrew']
2014-01-02|2         |CC         |['Andrew'] 
2014-01-03|2         |CC         |['Andrew']
2014-01-04|3         |OI         |['Steve']
2014-01-05|4         |OI         |['Ray']
2014-01-06|3         |OI         |['Steve', 'Stef']

In [21]:
list=[[1, 'A', '01/20/2020'],
       [1, 'B', '01/25/2020'],
       [2, 'A', '02/20/2020'],
       [2, 'B', '02/25/2020'],
       [2, 'C', '02/25/2020']]


df=spark.createDataFrame(list,['id','test','date'])

df.show()

+---+----+----------+
 id|test| date|
+---+----+----------+
 1| A|01/20/2020|
 1| B|01/25/2020|
 2| A|02/20/2020|
 2| B|02/25/2020|
 2| C|02/25/2020|
+---+----+----------+

In [22]:
list=[[2, 'A', '02/20/2020'],
       [2, 'B', '02/25/2020'],
       [2, 'C', '02/25/2020']]


df=spark.createDataFrame(list,['id','test','date'])

df.show()

+---+----+----------+
 id|test| date|
+---+----+----------+
 2| A|02/20/2020|
 2| B|02/25/2020|
 2| C|02/25/2020|
+---+----+----------+

In [23]:
uniq_ids = df.select('id').distinct().coalesce(1)
uniq_tests = df.select('test').distinct().coalesce(1)
skeleton = (
    uniq_ids.
        crossJoin(
            uniq_tests
        )
)
(
    skeleton.
        join(
            df,
            ['id', 'test'],
            'left'
        ).
        orderBy('id', 'test', 'date').
        explain()
)

== Physical Plan ==
Sort [id#1206L ASC NULLS FIRST, test#1207 ASC NULLS FIRST, date#1339 ASC NULLS FIRST], true, 0
+- Exchange rangepartitioning(id#1206L ASC NULLS FIRST, test#1207 ASC NULLS FIRST, date#1339 ASC NULLS FIRST, 200), [id=#1110]
 +- *(6) Project [id#1206L, test#1207, date#1339]
 +- SortMergeJoin [id#1206L, test#1207], [id#1337L, test#1338], LeftOuter
 :- Sort [id#1206L ASC NULLS FIRST, test#1207 ASC NULLS FIRST], false, 0
 : +- Exchange hashpartitioning(id#1206L, test#1207, 200), [id=#1101]
 : +- CartesianProduct
 : :- Coalesce 1
 : : +- *(2) HashAggregate(keys=[id#1206L], functions=[])
 : : +- Exchange hashpartitioning(id#1206L, 200), [id=#1089]
 : : +- *(1) HashAggregate(keys=[id#1206L], functions=[])
 : : +- *(1) Project [id#1206L]
 : : +- *(1) Scan ExistingRDD[id#1206L,test#1207,date#1208]
 : +- Coalesce 1
 : +- *(4) HashAggregate(keys=[test#1207], functions=[])
 : +- Exchange hashpartitioning(test#1207, 200), [id=#1095]
 : +- *(3) HashAggregate(keys=[test#1207], functions=[])
 : +- *(3) Project [test#1207]
 : +- *(3) Scan ExistingRDD[id#1206L,test#1207,date#1208]
 +- Sort [id#1337L ASC NULLS FIRST, test#1338 ASC NULLS FIRST], false, 0
 +- Exchange hashpartitioning(id#1337L, test#1338, 200), [id=#1104]
 +- *(5) Filter (isnotnull(id#1337L) && isnotnull(test#1338))
 +- *(5) Scan ExistingRDD[id#1337L,test#1338,date#1339]

In [24]:
df.groupBy("id").agg(F.collect_list("test").alias("x"),F.collect_list("date").alias("col2"))\
               .withColumn("zip", F.arrays_zip(F.col("x"),F.col("col2")))\
               .withColumn("except", F.array_except(F.array(*(F.lit(x) for x in ['A','B','C'])),"x")).drop("x","col2")\
               .withColumn("except", F.expr("""transform(except,x-> struct(x,'NA'))"""))\
               .withColumn("zipped", F.explode(F.array_union("zip","except")))\
               .select("id",F.col("zipped.x").alias("test"),F.col("zipped.col2").alias("date"))\
               .explain()

== Physical Plan ==
*(2) Project [id#1206L, zipped#1400.x AS test#1405, zipped#1400.col2 AS date#1406]
+- *(2) Generate explode(array_union(zip#1380, except#1394)), [id#1206L], false, [zipped#1400]
 +- *(2) Project [id#1206L, arrays_zip(x#1374, col2#1376) AS zip#1380, transform(array_except([A,B,C], x#1374), lambdafunction(named_struct(x, lambda x#1395, col2, NA), lambda x#1395, false)) AS except#1394]
 +- ObjectHashAggregate(keys=[id#1206L], functions=[collect_list(test#1207, 0, 0), collect_list(date#1208, 0, 0)])
 +- Exchange hashpartitioning(id#1206L, 200), [id=#1192]
 +- *(1) Project [id#1206L, date#1208, test#1207]
 +- *(1) Scan ExistingRDD[id#1206L,test#1207,date#1208]

In [26]:
list=[['B', 'UES', 5,1],
       ['B', 'MID', 10,2],
       ['B', 'UWS', 4,3],
       ['BR', 'EV', 1,1],
       ['BR', 'WB', 4,2],
       ['BR', 'MID', 5,3]]

df=spark.createDataFrame(list,['Borough','Neighborhood','Count','Row_Number'])

df.show()

+-------+------------+-----+----------+
Borough|Neighborhood|Count|Row_Number|
+-------+------------+-----+----------+
 B| UES| 5| 1|
 B| MID| 10| 2|
 B| UWS| 4| 3|
 BR| EV| 1| 1|
 BR| WB| 4| 2|
 BR| MID| 5| 3|
+-------+------------+-----+----------+

In [27]:
df.groupBy("Borough").pivot("Neighborhood").agg(F.first("Count")).show()

+-------+----+---+----+----+----+
Borough| EV|MID| UES| UWS| WB|
+-------+----+---+----+----+----+
 B|null| 10| 5| 4|null|
 BR| 1| 5|null|null| 4|
+-------+----+---+----+----+----+

In [28]:
from pyspark.sql import functions as F
from pyspark.sql.window import Window

df.withColumn("neighbour_dummy", F.concat_ws("_", F.lit("Neighbourhood"), F.col("Row_Number")))\
  .groupBy("Borough").pivot("neighbour_dummy").agg(F.first("Neighborhood").alias(""),\
                                                  F.first("count").alias("count")).show()

+-------+----------------+---------------------+----------------+---------------------+----------------+---------------------+
Borough|Neighbourhood_1_|Neighbourhood_1_count|Neighbourhood_2_|Neighbourhood_2_count|Neighbourhood_3_|Neighbourhood_3_count|
+-------+----------------+---------------------+----------------+---------------------+----------------+---------------------+
 B| UES| 5| MID| 10| UWS| 4|
 BR| EV| 1| WB| 4| MID| 5|
+-------+----------------+---------------------+----------------+---------------------+----------------+---------------------+

In [29]:
from pyspark.sql import functions as F
from pyspark.sql.window import Window

df.withColumn("row_num", F.row_number().over(Window.partitionBy("Borough").orderBy("Count"))).show()

+-------+------------+-----+-------+---------------+
Borough|Neighborhood|Count|row_num|neighbour_dummy|
+-------+------------+-----+-------+---------------+
 B| UWS| 4| 1| Neighbour_1|
 B| UES| 5| 2| Neighbour_2|
 B| MID| 10| 3| Neighbour_3|
 BR| EV| 1| 1| Neighbour_1|
 BR| WB| 4| 2| Neighbour_2|
 BR| MID| 5| 3| Neighbour_3|
+-------+------------+-----+-------+---------------+

In [30]:
pivot_neigh.show()

+-------+-----------+-----------+-----------+
Borough|Neighbour_1|Neighbour_2|Neighbour_3|
+-------+-----------+-----------+-----------+
 B| UWS| UES| MID|
 BR| EV| WB| MID|
+-------+-----------+-----------+-----------+

In [31]:
list=[['Jan-17']]

df=spark.createDataFrame(list,['t'])

df.show()

+------+
 t|
+------+
Jan-17|
+------+

In [32]:
from pyspark.sql.functions import *
df.select(to_timestamp('t',"MMM-yy HH:mm:ss").alias("dt")).show()

+----+
 dt|
+----+
null|
+----+

In [33]:
list=[[0,'Town Street','01-07-2017 08:08:00'],
      [0,'Town Street','01-07-2017 08:13:00'],
      [0,'Town Street','01-07-2017 08:18:00'],
      [0,'Town Street','01-07-2017 08:27:00'],
      [0,'Town Street','01-07-2017 08:45:00'],
      [0,'Town Street','01-07-2017 08:49:00'],
      [0,'Town Street','01-08-2017 08:10:00'],
      [0,'Town Street','01-08-2017 08:15:00']]

df=spark.createDataFrame(list,['col1','col2','timestamp'])

df.show()

+----+-----------+-------------------+
col1| col2| timestamp|
+----+-----------+-------------------+
 0|Town Street|01-07-2017 08:08:00|
 0|Town Street|01-07-2017 08:13:00|
 0|Town Street|01-07-2017 08:18:00|
 0|Town Street|01-07-2017 08:27:00|
 0|Town Street|01-07-2017 08:45:00|
 0|Town Street|01-07-2017 08:49:00|
 0|Town Street|01-08-2017 08:10:00|
 0|Town Street|01-08-2017 08:15:00|
+----+-----------+-------------------+

In [34]:
w=Window().partitionBy("date").orderBy(F.col("timestamp"))
w2=Window().partitionBy("date","interval")
w3=Window().partitionBy("date","interval").orderBy(F.col("timestamp"))

df.withColumn("timestamp", F.to_timestamp("timestamp",'MM-dd-yyyy HH:mm:ss'))\
  .withColumn("date", F.to_date("timestamp"))\
  .withColumn("lag", F.lag("timestamp").over(w))\
  .withColumn("diff", F.when(F.col("lag").isNotNull(),F.col("timestamp").cast("long")-F.col("lag").cast("long"))\
                       .otherwise(F.lit(None)))\
  .withColumn("interval",F.sum(F.when(F.col("diff")/300>1, F.lit(1)).otherwise(F.lit(0))).over(w))\
  .withColumn("collect", F.collect_list("timestamp").over(w2)).drop("lag","diff")\
  .withColumn("rownum",F.row_number().over(w3)).filter('rownum=1 and size(collect)>1')\
  .select("date", F.array(F.date_format("timestamp", "HH:mm:ss"),F.size("collect")).alias("Time"))\
  .orderBy("date").show(truncate=False)

+----------+-------------+
date |Time |
+----------+-------------+
2017-01-07|[08:08:00, 3]|
2017-01-07|[08:45:00, 2]|
2017-01-08|[08:10:00, 2]|
+----------+-------------+

In [35]:
w=Window().partitionBy("date").orderBy(F.col("timestamp"))
w2=Window().partitionBy("date","interval")
w3=Window().partitionBy("date","interval").orderBy(F.col("timestamp"))

df.withColumn("timestamp", F.to_timestamp("timestamp",'MM-dd-yyyy HH:mm:ss'))\
  .withColumn("date", F.to_date("timestamp"))\
  .withColumn("lag", F.lag("timestamp").over(w))\
  .withColumn("diff", F.when(F.col("lag").isNotNull(),F.col("timestamp").cast("long")-F.col("lag").cast("long"))\
                       .otherwise(F.lit(None)))\
  .withColumn("interval",F.sum(F.when(F.col("diff")/300>1, F.lit(1)).otherwise(F.lit(0))).over(w))\
  .show()

+----+-----------+-------------------+----------+-------------------+----+--------+
col1| col2| timestamp| date| lag|diff|interval|
+----+-----------+-------------------+----------+-------------------+----+--------+
 0|Town Street|2017-01-08 08:10:00|2017-01-08| null|null| 0|
 0|Town Street|2017-01-08 08:15:00|2017-01-08|2017-01-08 08:10:00| 300| 0|
 0|Town Street|2017-01-07 08:08:00|2017-01-07| null|null| 0|
 0|Town Street|2017-01-07 08:13:00|2017-01-07|2017-01-07 08:08:00| 300| 0|
 0|Town Street|2017-01-07 08:18:00|2017-01-07|2017-01-07 08:13:00| 300| 0|
 0|Town Street|2017-01-07 08:27:00|2017-01-07|2017-01-07 08:18:00| 540| 1|
 0|Town Street|2017-01-07 08:45:00|2017-01-07|2017-01-07 08:27:00|1080| 2|
+----+-----------+-------------------+----------+-------------------+----+--------+